In [14]:
import scipy.stats as stats
import pingouin as pg
import pandas as pd

In [15]:
df_excel = pd.read_excel("./WelchTest_GooglePlay_dataset.xlsx", "ANOVA_CATEGORIES_RATINGS")

In [16]:
df_excel

,COMMUNICATION,FAMILY,GAME,PRODUCTIVITY
0,4.3,4.1,3.8,4.4
1,4.2,4.2,4.5,3.4
2,4.3,4.3,4.5,4.1
3,4.0,0.5,4.0,4.1
4,4.3,4.3,4.3,3.6
...,...,...,...,...
506,NaN,4.5,NaN,NaN
507,NaN,1.9,NaN,NaN
508,NaN,4.6,NaN,NaN
509,NaN,4.3,NaN,NaN


In [17]:
df_excel.describe()

,COMMUNICATION,FAMILY,GAME,PRODUCTIVITY
count,97.000000,511.000000,302.000000,93.000000
mean,4.141237,4.176223,4.271523,4.221505
std,0.374931,0.545433,0.373789,0.490308
min,2.300000,0.500000,1.400000,1.900000
25%,4.000000,4.000000,4.100000,4.100000
50%,4.200000,4.300000,4.400000,4.300000
75%,4.400000,4.500000,4.500000,4.500000
max,4.800000,5.000000,5.000000,5.000000


In [18]:
# Data for each group
communicationSize = df_excel["COMMUNICATION"]
familySize = df_excel["FAMILY"]
gameSize = df_excel["GAME"]
productivitySize = df_excel["PRODUCTIVITY"]

In [19]:
# Drop NaN values from the group arrays
communicationSize = communicationSize.dropna()
familySize = familySize.dropna()
gameSize = gameSize.dropna()
productivitySize = productivitySize.dropna()

In [20]:
# Perform ANOVA
f_value, p_value = stats.f_oneway(communicationSize, familySize, gameSize, productivitySize)
print("ANOVA results:")
print("F-value:", f_value)
print("p-value:", p_value)

ANOVA results:
F-value: 3.1628809495541423
p-value: 0.023884668463414905


### EXCEL ANOVA TEST = 0,0281327317086259
### Næste gang = 0,0238846684661909

In [21]:
# perform Bartlett's test
stats.bartlett(communicationSize, familySize, gameSize,productivitySize)

BartlettResult(statistic=60.17323284264685, pvalue=5.398035992544165e-13)

### EXCEL Test for varianshomogenitet = 2,95685308899291E-10
### Næste gang = 5,39803599257789E-13

In [22]:
df = df_excel.stack().reset_index()

In [23]:
df.columns = ['idx','categories', 'values']

In [24]:
df = df.drop('idx', axis=1)

In [25]:
df

,categories,values
0,COMMUNICATION,4.3
1,FAMILY,4.1
2,GAME,3.8
3,PRODUCTIVITY,4.4
4,COMMUNICATION,4.2
...,...,...
998,FAMILY,4.5
999,FAMILY,1.9
1000,FAMILY,4.6
1001,FAMILY,4.3


In [26]:
# Perform one-way ANOVA
anova = pg.anova(data=df, dv='values', between='categories', detailed=True)
anova

,Source,SS,DF,MS,F,p-unc,np2
0,categories,2.178786,3,0.726262,3.162881,0.023885,0.009409
1,Within,229.390751,999,0.229620,NaN,NaN,NaN


In [27]:
# perform Welch's ANOVA
Wanova = pg.welch_anova(dv='values', between='categories', data=df)
Wanova

,Source,ddof1,ddof2,F,p-unc,np2
0,categories,3,265.247149,4.394193,0.004882,0.009409


# Hvor ligger forskellen?

In [28]:
# Perform Welch's t-test between pairs of groups
t_value, p_value = stats.ttest_ind(communicationSize, familySize, equal_var=False)
print("\nWelch's t-test between communicationSize and familySize:")
print("t-value:", t_value)
print("p-value:", p_value)


Welch's t-test between communicationSize and familySize:
t-value: -0.7762410202612958
p-value: 0.4386084334814365


In [29]:
t_value, p_value = stats.ttest_ind(communicationSize, gameSize, equal_var=False)
print("\nWelch's t-test between communicationSize and gameSize:")
print("t-value:", t_value)
print("p-value:", p_value)


Welch's t-test between communicationSize and gameSize:
t-value: -2.979687476784126
p-value: 0.003330442399738894


### forskellen ligger mellem communicationSize og productivitySize

In [30]:
t_value, p_value = stats.ttest_ind(communicationSize, productivitySize, equal_var=False)
print("\nWelch's t-test between communicationSize and productivitySize:")
print("t-value:", t_value)
print("p-value:", p_value)


Welch's t-test between communicationSize and productivitySize:
t-value: -1.2637648469654237
p-value: 0.20802254964150274


In [31]:
t_value, p_value = stats.ttest_ind(familySize, gameSize, equal_var=False)
print("\nWelch's t-test between familySize and gameSize:")
print("t-value:", t_value)
print("p-value:", p_value)


Welch's t-test between familySize and gameSize:
t-value: -2.94829383616811
p-value: 0.0032890965769201566


In [32]:
t_value, p_value = stats.ttest_ind(familySize, productivitySize, equal_var=False)
print("\nWelch's t-test between familySize and productivitySize:")
print("t-value:", t_value)
print("p-value:", p_value)


Welch's t-test between familySize and productivitySize:
t-value: -0.8046252961961513
p-value: 0.4224315353376734


In [33]:
t_value, p_value = stats.ttest_ind(gameSize, productivitySize, equal_var=False)
print("\nWelch's t-test between gameSize and productivitySize:")
print("t-value:", t_value)
print("p-value:", p_value)


Welch's t-test between gameSize and productivitySize:
t-value: 0.9060347899149139
p-value: 0.366638078620661


# Tukey test

In [34]:
# Conduct post-hoc Tukey HSD test
posthoc = pg.pairwise_tukey(dv='values', between='categories', data=df)

In [35]:
posthoc

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,COMMUNICATION,FAMILY,4.141237,4.176223,-0.034986,0.053071,-0.659225,0.912299,-0.066921
1,COMMUNICATION,GAME,4.141237,4.271523,-0.130286,0.055925,-2.329677,0.092042,-0.347639
2,COMMUNICATION,PRODUCTIVITY,4.141237,4.221505,-0.080268,0.069543,-1.154222,0.655899,-0.183690
3,FAMILY,GAME,4.176223,4.271523,-0.095300,0.034781,-2.740039,0.031709,-0.194782
4,FAMILY,PRODUCTIVITY,4.176223,4.221505,-0.045282,0.054022,-0.838217,0.836235,-0.084161
5,GAME,PRODUCTIVITY,4.271523,4.221505,0.050018,0.056828,0.880168,0.815166,0.123543


In [36]:
# Print the significant pairwise comparisons
significant_comparisons = posthoc[posthoc['p-tukey'] < 0.05]
print("Significant Comparisons:")
print(significant_comparisons[['A', 'B', 'mean(A)', 'mean(B)', 'p-tukey']])

Significant Comparisons:
        A     B   mean(A)   mean(B)   p-tukey
3  FAMILY  GAME  4.176223  4.271523  0.031709
